In [19]:
from keras.models import Sequential
from keras.layers import MaxPooling1D, Conv1D, Embedding, LSTM, Flatten, Dense, Dropout

import pandas as pd
import os
import numpy as np
import re

In [20]:
def normalize_signal(signal):
    """
    time_series
    
    Returns normalized TS: (TS - mean(TS))/std(TS)
    """
    std = np.std(signal)
    norm_signal = (signal - np.mean(signal)) / std if std else 1
    return norm_signal

In [21]:
def normalize_all_signals_in_ds(one_group_ds):
    for signal in one_group_ds.drop('t', axis=1):
        one_group_ds[signal] = normalize_signal(one_group_ds[signal])

In [22]:
def append_signals_dataset(data_directory, signals_dataset = None, labels = None):
    data_dir_list = os.listdir(data_directory)
    os.chdir(data_directory)
    signal_id = 0
    if not labels:
        labels = pd.DataFrame(columns=['id', 'class', 'label'])
    for signal_label, directory in enumerate(data_dir_list):
        if os.path.isfile(directory) or directory[0] == '.':
                continue
        os.chdir(directory)
        for filename in os.listdir(os.curdir):
            if re.match(r'(.*csv)', filename):
                if (signals_dataset) is not None:
                    tmp_df = pd.read_csv(filename)
                    normalize_all_signals_in_ds(tmp_df)
                    tmp_df['id'] = signal_id
                    signals_dataset = signals_dataset.append(tmp_df, ignore_index=True)
                else:
                    signals_dataset = pd.read_csv(filename)
                    normalize_all_signals_in_ds(signals_dataset)
                    signals_dataset['id'] = 0
#                 print(signals_dataset.head(5))
                tmp_labels = pd.Series([signal_id, directory, np.int(signal_label)],
                                         index=labels.columns)
                labels = labels.append(tmp_labels, ignore_index=True)
                signal_id += 1
        os.chdir('../')
    os.chdir('../..')
    return signals_dataset, labels

In [181]:
def normalize_labels(labels_df):
    """
    Разметим все классы числами от 0 до количества классов
    и преобразуем новую колонку в формат чисел (float)
    """
    unique_classes = np.unique(labels_df['class'])
    labels_df['label'] = labels['class'].map(dict(zip(unique_classes, range(len(unique_classes)))))
    labels_df.label = pd.to_numeric(labels.label)
    labels_df.id = pd.to_numeric(labels_df.id)

In [182]:
# os.chdir('..')
os.getcwd()

'/Users/aolen/master_degree'

In [183]:
dataset, labels = append_signals_dataset('./data/gen_1_25')

In [184]:
dataset_t = dataset.T

In [185]:
dataset_t

,0,1,2,3,4,5,6,7,8,9,...,7427,7428,7429,7430,7431,7432,7433,7434,7435,7436
t,0.010000,0.020000,0.030000,0.040000,0.050000,0.060000,0.070000,0.080000,0.090000,0.100000,...,1.910000,1.920000,1.930000,1.940000,1.950000,1.960000,1.970000,1.980000,1.990000,2.000000
U,0.792477,0.792477,0.792477,0.792477,0.792477,0.792477,0.792477,0.792477,0.792477,0.792477,...,-1.146624,-1.140144,-1.133665,-1.127186,-1.114228,-1.107748,-1.094790,-1.088311,-1.075353,-1.068873
U.1,1.131912,1.131912,1.131912,1.131912,1.131912,1.131912,1.131912,1.131912,1.131912,1.131912,...,-1.087143,-1.087143,-1.087143,-1.087143,-1.087143,-1.087143,-1.087143,-1.040221,-1.040221,-1.040221
Pij,-2.535839,-2.535839,-2.535839,-2.535839,-2.535839,-2.535839,-2.535839,-2.535839,-2.535839,-2.535839,...,0.540917,0.542015,0.543113,0.544210,0.545308,0.546772,0.547869,0.549333,0.550431,0.551894
Pg,1.576293,1.576293,1.576293,1.576293,1.576293,1.576293,1.576293,1.576293,1.576293,1.576293,...,0.864676,0.864676,0.863888,0.863888,0.863101,0.863101,0.862314,0.861527,0.860739,0.859952
Pgt,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
delta,-1.077129,-1.077129,-1.077129,-1.077129,-1.077129,-1.077129,-1.077129,-1.077129,-1.077129,-1.077129,...,1.108177,1.102083,1.095312,1.087864,1.080416,1.072291,1.064166,1.056041,1.046562,1.037760
id,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000


In [186]:
pd.to_numeric(np.unique(dataset.t))

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1 ,
       1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 , 1.21,
       1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31, 1.32,
       1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42, 1.43,
       1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.5 , 1.

In [187]:
pd.to_numeric(np.unique(dataset.t) * 100, downcast='unsigned').reshape(-1)

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148., 149., 150., 15

In [188]:
U_signals = pd.DataFrame(columns=np.arange(0,201))
U_signals

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200


In [189]:
signal_len = 201
for i, id in enumerate(np.unique(dataset.id)):
    
    temp = np.array(dataset[dataset['id'] == id].loc[:,['U']].T)
    U_signals = U_signals.append(pd.Series(temp[0]), ignore_index=True)

In [190]:
normalize_labels(labels)
labels

,id,class,label
0,0,le_8,0
1,1,le_8,0
2,2,le_8,0
3,3,le_8,0
4,4,le_8,0
5,5,le_8,0
6,6,le_8,0
7,7,le_8,0
8,8,le_8,0
9,9,le_8,0


In [194]:
U_ts_labeled = U_signals.merge(labels.label, right_on=labels.id, left_on=U_signals.index).drop('key_0', axis=1)

In [200]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [201]:
X_train, X_test, y_train, y_test = train_test_split(U_signals, labels.label, test_size=0.2, random_state=42)

In [288]:
X_train.shape

(29, 201)

In [206]:
TIME_PERIODS = 201

In [291]:
X_train_3d = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)

In [292]:
X_test_3d = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

In [381]:
model = Sequential()
model.add(Conv1D(32, 10, padding='same', activation='relu', input_shape=(TIME_PERIODS,1)))
model.add(MaxPooling1D(pool_size=10))
model.add(Conv1D(64, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=10))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [326]:
model = Sequential()
model.add(Dense(201, input_dim=201))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])

In [382]:
model

In [383]:
model.fit(X_train_3d, y_train, epochs=6)

Epoch 1/6
29/29 [==============================] - 1s 37ms/step - loss: 0.6805 - accuracy: 0.5862
Epoch 2/6
29/29 [==============================] - 0s 433us/step - loss: 0.6752 - accuracy: 0.6207
Epoch 3/6
29/29 [==============================] - 0s 557us/step - loss: 0.6606 - accuracy: 0.6897
Epoch 4/6
29/29 [==============================] - 0s 513us/step - loss: 0.6493 - accuracy: 0.7586
Epoch 5/6
29/29 [==============================] - 0s 547us/step - loss: 0.6255 - accuracy: 0.8276
Epoch 6/6
29/29 [==============================] - 0s 526us/step - loss: 0.6224 - accuracy: 0.7931


In [384]:
model.metrics_names

['loss', 'accuracy']

In [385]:
model.evaluate(X_test_3d, y_test)

8/8 [==============================] - 0s 27ms/step


[0.5827293395996094, 0.875]

In [278]:
import keras.metrics

In [ ]:
keras.metrics.accuracy